# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [3]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Create complete Model with Custom output Layer

In [4]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense (Dense)               (None, 100)               100100    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data
Be sure to change file contents

In [5]:
trainingValues = [0] * 612 + [1] * 719
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# Test Images not configured
# testImages = keras.utils.image_dataset_from_directory(
#     directory='testData',
#     labels= testValues,
#     label_mode='binary',
#     batch_size=32,
#     image_size=(224, 224))
# print (trainImages)

Found 1331 files belonging to 2 classes.


## Prediction before training

In [6]:
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-11-06 16:23:39.265008: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-06 16:23:39.370765: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 288ms/step
[[0.5159187]]


## Train Last two layers

In [7]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-11-06 16:23:39.974934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


42/42 [==============================] - 14s 316ms/step - loss: 0.6687 - accuracy: 0.6093
Epoch 2/10
42/42 [==============================] - 13s 313ms/step - loss: 0.5840 - accuracy: 0.7979
Epoch 3/10
42/42 [==============================] - 13s 313ms/step - loss: 0.4713 - accuracy: 0.8971
Epoch 4/10
42/42 [==============================] - 13s 313ms/step - loss: 0.3765 - accuracy: 0.9061
Epoch 5/10
42/42 [==============================] - 13s 313ms/step - loss: 0.3110 - accuracy: 0.9174
Epoch 6/10
42/42 [==============================] - 13s 313ms/step - loss: 0.2667 - accuracy: 0.9234
Epoch 7/10
42/42 [==============================] - 13s 313ms/step - loss: 0.2350 - accuracy: 0.9256
Epoch 8/10
42/42 [==============================] - 13s 313ms/step - loss: 0.2113 - accuracy: 0.9339
Epoch 9/10
42/42 [==============================] - 13s 313ms/step - loss: 0.1929 - accuracy: 0.9384
Epoch 10/10
42/42 [==============================] - 13s 313ms/step - loss: 0.1782 - accuracy: 0.9399


## Test Some data

In [8]:
# score = completeModel.evaluate(testImages)
# print ("loss = ", score[0])
# print ("accuracy = ", score[1])

## See what happens with a deer image

In [9]:
from PIL import Image
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer, 1 is fox, 2 is Possum, 3 is rabbit, 4 is squirrel

1/1 [==============================] - 0s 22ms/step
[[0.9972402]]


## Save the model for export

In [10]:
completeModel.save("mouseModel.keras")

## Check test data for each animal

In [11]:
# import required module
import os
# assign directory
directory = 'extraTest/testDeer'
 
# iterate over files in
# that directory
# 0 is positives 1 is negativess
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

extraTest/testDeer/1705.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.85394377]]
extraTest/testDeer/1687.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9948832]]
extraTest/testDeer/1703.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.87328565]]
extraTest/testDeer/1692.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.99011713]]
extraTest/testDeer/1673.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.991122]]
extraTest/testDeer/1694.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9864144]]
extraTest/testDeer/1681.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.99723023]]
extraTest/testDeer/1679.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9962238]]
extraTest/testDeer/1698.jpg__

In [12]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/3282.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg


1/1 [==============================] - 0s 15ms/step
[[0.991678]]
extraTest/testFox/3295.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9837705]]
extraTest/testFox/3278.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.99033207]]
extraTest/testFox/3294.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.96949875]]
extraTest/testFox/3287.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.5245131]]
extraTest/testFox/3297.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9645229]]
extraTest/testFox/3281.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.91741794]]
extraTest/testFox/3292.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9293718]]
extraTest/testFox/3279.jpg___crop00_md_v5a.0.0.pt.jpg
1/

In [13]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.97960585]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.8201101]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.98311216]]
extraTest/testPossum/1937.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.99149245]]
extraTest/testPossum/1995.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.8312786]]
extraTest/testPossum/1986.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9979512]]
extraTest/testPossum/1974.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9982382]]
extraTest/testPossum/1967.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.99539113]]
extraTest/te

In [14]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg


1/1 [==============================] - 0s 18ms/step
[[0.65936214]]
extraTest/testRabbit/765.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.5076576]]
extraTest/testRabbit/776.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9375129]]
extraTest/testRabbit/770.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.6954037]]
extraTest/testRabbit/763.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.8851352]]
extraTest/testRabbit/758.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.61418134]]
extraTest/testRabbit/741.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.8756741]]
extraTest/testRabbit/752.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 18ms/step
[[0.89512193]]
extraTest/testRabbit/754.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [================

In [15]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.10403786]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.69538856]]
extraTest/testSquirrel/118.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.41160497]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.22773454]]
extraTest/testSquirrel/125.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9381084]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.6963493]]
extraTest/testSquirrel/122.jpg___crop03_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.61604077]]
extraTest/testSquirrel/101.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.03540779]]
extraT

In [16]:
directory = 'extraTest/testDog'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testDog/5654.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.264328]]
extraTest/testDog/5645.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.91398287]]
extraTest/testDog/5657.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9978231]]
extraTest/testDog/5648.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.98306805]]
extraTest/testDog/5651.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.97545165]]
extraTest/testDog/5649.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.9224986]]
extraTest/testDog/5656.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 16ms/step
[[0.9909656]]
extraTest/testDog/5655.jpg

In [17]:
directory = 'extraTest/testCat'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        # newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.98784685]]
extraTest/testCat/7043.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9936783]]
extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9885346]]
extraTest/testCat/7044.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.8229914]]
extraTest/testCat/7045.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.4316141]]
extraTest/testCat/7048.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9876003]]
extraTest/testCat/7045.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.13057597]]
extraTest/testCat/7047.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [===

In [18]:
directory = 'extraTest/testCoyote'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        # newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCoyote/58ac0d93-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9532107]]
extraTest/testCoyote/58b2edb2-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.99236286]]
extraTest/testCoyote/5a049367-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.9685033]]
extraTest/testCoyote/5a096aa5-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.890559]]
extraTest/testCoyote/58b9d752-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9915479]]
extraTest/testCoyote/58b9d6cb-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.9939314]]
extraTest/

In [19]:
directory = 'extraTest/testWeasel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        # newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testWeasel/image_01408.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.8842648]]
extraTest/testWeasel/image_01071.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.32367662]]
extraTest/testWeasel/image_01072.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.16834763]]
extraTest/testWeasel/image_01072.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.31847087]]
extraTest/testWeasel/image_00821.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.9671087]]
extraTest/testWeasel/image_00821.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 15ms/step
[[0.94937134]]
extraTest/testWeasel/image_01408.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.28689283]]
extraTest/testWeasel/imag

## Code for data manipulation to creaet more data

In [21]:
directory = 'extraTest/testMouse'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        # newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testMouse/0071.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.3389118]]
extraTest/testMouse/0062.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.25506166]]
extraTest/testMouse/0072.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.09823373]]
extraTest/testMouse/0100.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.15072206]]
extraTest/testMouse/0073.jpg___crop01_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.24250667]]
extraTest/testMouse/0071.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.2839358]]
extraTest/testMouse/0100.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.07316004]]
extraTest/testMouse/0074 (2).jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [================

In [20]:
# import os
# from PIL import Image
# directory = 'extraTraining/cropped_mouse' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])